In [1]:
from scipy.io import savemat, loadmat
import mat73
import hdf5storage as st
import pickle
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.sparse.linalg import eigsh
from sklearn.decomposition import PCA
from scipy.stats import wilcoxon
from scipy.optimize import curve_fit
import seaborn as sns
from copy import deepcopy as dc
from statsmodels.stats.multitest import multipletests

from sklearn import svm
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split, KFold, StratifiedKFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from itertools import combinations
import math

# # tell pandas to show all columns when we display a DataFrame
# pd.set_option("display.max_columns", None)

In [ ]:
# loading variables

# openscope
with open('SVM_prerequisite_variables.pickle', 'rb') as f:
    SVM_prerequisite_variables = pickle.load(f)
    
    list_rate_w1 = SVM_prerequisite_variables['list_rate_w1'].copy()
    list_stm_w1 = SVM_prerequisite_variables['list_stm_w1'].copy()
    list_neu_loc = SVM_prerequisite_variables['list_neu_loc'].copy()
    list_wfdur = SVM_prerequisite_variables['list_wfdur'].copy()
    list_slopes_an_loglog = SVM_prerequisite_variables['list_slopes_an_loglog'].copy()
    list_slopes_an_loglog_12 = SVM_prerequisite_variables['list_slopes_an_loglog_12'].copy()

num_sess = len(list_rate_w1)

# ABO Neuropixels
with open('resp_matrix_ep_RS_all_32sess_allensdk.pickle', 'rb') as f:
    resp_matrix_ep_RS_all = pickle.load(f)

    list_rate_RS = resp_matrix_ep_RS_all['list_rate_RS'].copy()
    list_rate_RS_dr = resp_matrix_ep_RS_all['list_rate_RS_dr'].copy()
    list_rate_all = resp_matrix_ep_RS_all['list_rate_all'].copy()
    list_rate_all_dr = resp_matrix_ep_RS_all['list_rate_all_dr'].copy()
    list_slopes_RS_an_loglog = resp_matrix_ep_RS_all['list_slopes_RS_an_loglog'].copy()
    list_slopes_all_an_loglog = resp_matrix_ep_RS_all['list_slopes_all_an_loglog'].copy()

    sess_inds_qual_all = resp_matrix_ep_RS_all['sess_inds_qual_all'].copy()
    sess_inds_qual_all_dr = resp_matrix_ep_RS_all['sess_inds_qual_all_dr'].copy()

num_sess_ABO = len(list_rate_all)

In [3]:
def compute_mean_var_trial(label_cnt_dict, rate_sorted):    
    list_trial_mean = [[0]] * len(label_cnt_dict)
    list_trial_var = [[0]] * len(label_cnt_dict)

    for trial_ind, trial_type in enumerate(label_cnt_dict):
        
        trial_rate = np.array(rate_sorted.loc[:, trial_type])                
        trial_mean = np.mean(trial_rate, axis=1)
        trial_var = np.var(trial_rate, axis=1, ddof=1)

        trial_mean = pd.DataFrame(trial_mean, columns=[trial_type], index=rate_sorted.index)
        trial_var = pd.DataFrame(trial_var, columns=[trial_type], index=rate_sorted.index)
        list_trial_mean[trial_ind] = pd.concat([trial_mean] * label_cnt_dict[trial_type], axis=1)
        list_trial_var[trial_ind] = pd.concat([trial_var] * label_cnt_dict[trial_type], axis=1)

    rate_sorted_mean = pd.concat(list_trial_mean, axis=1)
    rate_sorted_var = pd.concat(list_trial_var, axis=1)

    return rate_sorted_mean, rate_sorted_var

In [4]:
def compute_mean_var_trial_collapse(label_cnt_dict, rate_sorted):    
    list_trial_mean = [[0]] * len(label_cnt_dict)
    list_trial_var = [[0]] * len(label_cnt_dict)

    for trial_ind, trial_type in enumerate(label_cnt_dict):
        
        trial_rate = np.array(rate_sorted.loc[:, trial_type])                
        trial_mean = np.mean(trial_rate, axis=1, dtype=np.longdouble)
        trial_var = np.var(trial_rate, axis=1, ddof=1, dtype=np.longdouble)

        trial_mean = pd.DataFrame(trial_mean, columns=[trial_type], index=rate_sorted.index)
        trial_var = pd.DataFrame(trial_var, columns=[trial_type], index=rate_sorted.index)
        list_trial_mean[trial_ind] = trial_mean.copy()
        list_trial_var[trial_ind] = trial_var.copy()

    rate_sorted_mean = pd.concat(list_trial_mean, axis=1)
    rate_sorted_var = pd.concat(list_trial_var, axis=1)

    return rate_sorted_mean, rate_sorted_var

In [ ]:
# # data loading (Neuropixels)

# list_rate_w1 = []
# list_stm_w1 = []

# list_rate_w0 = []
# list_stm_w0 = []

# list_rate_k1 = []
# list_stm_k1 = []

# list_rate_k0 = []
# list_stm_k0 = []

# list_neu_loc = []

# list_wfdur = []

# list_ep_folders = ['sub_619296', 'sub_620333', 'sub_620334', \
#     'sub_625545', 'sub_625554', 'sub_625555', 'sub_630506', 'sub_631510', \
#     'sub_631570', 'sub_633229', 'sub_637484']
# path_ep_folders = 'D:\\Users\\USER\\MATLAB\\OpenScopeData_00248_v240130_postprocessed\\'
# file_name_pp = '\\postprocessed.mat'
# file_name_qc = '\\qc_units.mat'

# for ep_folder in list_ep_folders:
#     pp = mat73.loadmat(path_ep_folders + ep_folder + file_name_pp)
#     qc = loadmat(path_ep_folders + ep_folder + file_name_qc)

#     # data loading
#     rate_w1 = np.squeeze(pp['Rall']['ICwcfg1_presentations'])
#     stm_w1 = np.squeeze(pp['vis']['ICwcfg1_presentations']['trialorder'])

#     rate_w0 = np.squeeze(pp['Rall']['ICwcfg0_presentations'])
#     stm_w0 = np.squeeze(pp['vis']['ICwcfg0_presentations']['trialorder'])

#     rate_k1 = np.squeeze(pp['Rall']['ICkcfg1_presentations'])
#     stm_k1 = np.squeeze(pp['vis']['ICkcfg1_presentations']['trialorder'])

#     rate_k0 = np.squeeze(pp['Rall']['ICkcfg0_presentations'])
#     stm_k0 = np.squeeze(pp['vis']['ICkcfg0_presentations']['trialorder'])

#     neualloc = np.squeeze(pp['neuallloc'])

#     wfdur = np.squeeze(qc['unit_wfdur'])

#     list_rate_w1.append(rate_w1)
#     list_stm_w1.append(stm_w1)

#     list_rate_w0.append(rate_w0)
#     list_stm_w0.append(stm_w0)

#     list_rate_k1.append(rate_k1)
#     list_stm_k1.append(stm_k1)

#     list_rate_k0.append(rate_k0)
#     list_stm_k0.append(stm_k0)

#     list_neu_loc.append(neualloc)

#     list_wfdur.append(wfdur)

#     print(ep_folder)

# num_sess = len(list_rate_w1)

In [6]:
# # Linear Fitting Slope across neurons (Ic Neuropixels) (loglog line)

# fig, axes = plt.subplots(1, 1, figsize=(5, 4))
# axes = np.array(axes).flatten()

# # Iterate over all sessions
# list_slopes_an_loglog = np.empty(num_sess, dtype=object)
# for sess_ind, (rate_all, stm, neu_loc, wfdur) in enumerate(zip(list_rate_w1, list_stm_w1, list_neu_loc, list_wfdur)):
#     print(f'session index: {sess_ind}')

#     # rate_all transposition
#     rate_all = rate_all.T.copy()

#     ser_neu_loc = pd.Series(neu_loc)

#     # Extract V1 neurons
#     list_vis = [ser_neu_loc.str.contains('VISp'), ~ser_neu_loc.str.contains('VISpm')]
#     list_vis = [all(bools) for bools in zip(*list_vis)]
#     rate = rate_all[list_vis].copy()
#     # list_visp_rs = [ser_neu_loc.str.contains('VISp'), ~ser_neu_loc.str.contains('VISpm'), (wfdur >= 0.4)]
#     # rate = rate_all[[all(bools) for bools in zip(*list_visp_rs)]].copy()
#     # print(np.sum([all(bools) for bools in zip(*list_visp_rs)]))

#     # Multiply by delta t to convert to spike counts
#     rate = rate * 0.4

#     # Create a counting dictionary for each stimulus
#     all_stm_unique, all_stm_counts = np.unique(stm, return_counts=True) 
#     stm_cnt_dict = dict(zip(all_stm_unique, all_stm_counts))
#     dict_trial_type = {0: 'Blank', 1: 'X', 2:'Tc1', 3: 'Ic1', 4: 'Lc1', 5:'Tc2', 6: 'Lc2', 7: 'Ic2', \
#         8: 'Ire1', 9: 'Ire2', 10: 'Tre1', 11: 'Tre2', 12: 'Xre1', 13: 'Xre2', 14: 'BR_in', 15: 'BL_in', \
#             16: 'TL_in', 17: 'TR_in', 18: 'BR_out', 19: 'BL_out', 20: 'TL_out', 21: 'TR_out'}
    
#     # Create label array for all stimuli
#     label = []
#     for i in stm:
#         for trial_type_num in dict_trial_type:
#             if i == trial_type_num:
#                 label.append(dict_trial_type[trial_type_num])
#     label = np.array(label)

#     # convert to dataframe
#     rate = pd.DataFrame(rate, columns=label)

#     # sort trials based on stimuli
#     rate_sorted = rate.sort_index(axis=1)
#     label_sorted = rate_sorted.columns.copy()

#     # Create a counting dictionary for each stimulus 
#     all_label_unique, all_label_counts = np.unique(label_sorted, return_counts=True) 
#     label_cnt_dict = dict(zip(all_label_unique, all_label_counts))

#     # Compute mean & variance for each stimulus
#     rate_sorted_mean, rate_sorted_var = compute_mean_var_trial(label_cnt_dict, rate_sorted)
#     rate_sorted_mean_coll, rate_sorted_var_coll = compute_mean_var_trial_collapse(label_cnt_dict, rate_sorted)

#     # Calculate and collect linear slopes for all stimuli
#     slopes = np.zeros((2, rate_sorted_mean_coll.shape[1]))
#     for trial_type_ind, trial_type in enumerate(rate_sorted_mean_coll.columns):
            
#         bool_mean_notzero = rate_sorted_mean_coll.loc[:, trial_type] > 0
#         popt = np.polyfit(np.log10(rate_sorted_mean.loc[bool_mean_notzero, trial_type].values).flatten().astype(np.float32), \
#                             np.log10(rate_sorted_var.loc[bool_mean_notzero, trial_type].values).flatten().astype(np.float32), 1)
#         # popt = np.polyfit(np.log10(rate_sorted_mean.loc[:, trial_type].values).flatten().astype(np.float32), \
#         #                     np.log10(rate_sorted_var.loc[:, trial_type].values).flatten().astype(np.float32), 1)
        
#         slopes[:, trial_type_ind] = popt.copy()
            
#     list_slopes_an_loglog[sess_ind] = slopes.copy()

# # Histogram of slopes (all sessions)
# # fig, ax = plt.subplots(figsize=(5, 4))

# list_slopes_an_loglog_flattened = np.concatenate([slopes[0, :].flatten() for slopes in list_slopes_an_loglog if slopes is not None])
# # list_slopes_an_loglog_flattened = np.concatenate((list_slopes_an_loglog_flattened[:1537], list_slopes_an_loglog_flattened[1538:]))

# bin_size = 0.01
# lower_bound, upper_bound = math.floor(np.min(list_slopes_an_loglog_flattened)), \
#     math.ceil(np.max(list_slopes_an_loglog_flattened))

# weights=np.ones_like(list_slopes_an_loglog_flattened)/len(list_slopes_an_loglog_flattened)
# axes[0].hist(list_slopes_an_loglog_flattened, bins=np.arange(lower_bound, upper_bound, bin_size), range=(lower_bound, upper_bound), \
#         weights=weights, color='cornflowerblue')
# # plt.axvline(1, color='r', linestyle='--')

# axes[0].set_xlabel('log(mean)-log(var) slope', fontsize=20)
# # if area_ind == 0 or area_ind == 3:
# axes[0].set_ylabel('Frequency of stimuli', fontsize=20)
# axes[0].set_xticks([0, 0.5, 1, 1.5, 2])
# # axes[0].set_yticks(np.arange(5, 40, 5))
# axes[0].tick_params('both', labelsize=18)

# # Wilcoxon signed-rank test
# wilc = list(wilcoxon(list_slopes_an_loglog_flattened, np.ones_like(list_slopes_an_loglog_flattened), alternative='greater'))
# # plt.annotate(f'p = {wilc[1]:.3f}', xy=(0.8, 0.9), xycoords='axes fraction')

# print(f'slope median = {np.median(list_slopes_an_loglog_flattened):.2f}')

# plt.subplots_adjust(hspace=0.4, wspace=0.3)
# plt.show()

In [7]:
# # Linear Fitting Slope across neurons (Ic Neuropixels) (loglog line) (high-repeat trial types)

# fig, axes = plt.subplots(1, 1, figsize=(5, 4))
# axes = np.array(axes).flatten()

# # Iterate over all sessions
# list_slopes_an_loglog_12 = np.empty(num_sess, dtype=object)
# for sess_ind, (rate_all, stm, neu_loc, wfdur) in enumerate(zip(list_rate_w1, list_stm_w1, list_neu_loc, list_wfdur)):
#     print(f'session index: {sess_ind}')

#     # rate_all transposition
#     rate_all = rate_all.T.copy()

#     ser_neu_loc = pd.Series(neu_loc)

#     # Extract V1 neurons
#     list_vis = [ser_neu_loc.str.contains('VISp'), ~ser_neu_loc.str.contains('VISpm')]
#     list_vis = [all(bools) for bools in zip(*list_vis)]
#     rate = rate_all[list_vis].copy()
#     # list_visp_rs = [ser_neu_loc.str.contains('VISp'), ~ser_neu_loc.str.contains('VISpm'), (wfdur >= 0.4)]
#     # rate = rate_all[[all(bools) for bools in zip(*list_visp_rs)]].copy()
#     # print(np.sum([all(bools) for bools in zip(*list_visp_rs)]))

#     # Multiply by delta t to convert to spike counts
#     rate = rate * 0.4

#     # Create a counting dictionary for each stimulus
#     all_stm_unique, all_stm_counts = np.unique(stm, return_counts=True) 
#     stm_cnt_dict = dict(zip(all_stm_unique, all_stm_counts))
#     dict_trial_type = {0: 'Blank', 1: 'X', 2:'Tc1', 3: 'Ic1', 4: 'Lc1', 5:'Tc2', 6: 'Lc2', 7: 'Ic2', \
#         8: 'Ire1', 9: 'Ire2', 10: 'Tre1', 11: 'Tre2', 12: 'Xre1', 13: 'Xre2', 14: 'BR_in', 15: 'BL_in', \
#             16: 'TL_in', 17: 'TR_in', 18: 'BR_out', 19: 'BL_out', 20: 'TL_out', 21: 'TR_out'}
    
#     # Create label array for all stimuli
#     label = []
#     for i in stm:
#         for trial_type_num in dict_trial_type:
#             if i == trial_type_num:
#                 label.append(dict_trial_type[trial_type_num])
#     label = np.array(label)

#     # convert to dataframe
#     rate = pd.DataFrame(rate, columns=label)

#     # sort trials based on stimuli
#     rate_sorted = rate.sort_index(axis=1)
#     label_sorted = rate_sorted.columns.copy()

#     # Create a counting dictionary for each stimulus 
#     all_label_unique, all_label_counts = np.unique(label_sorted, return_counts=True) 
#     label_cnt_dict = dict(zip(all_label_unique, all_label_counts))

#     # Extract stimuli having 400 repeats (trials)
#     rate_sorted = rate_sorted.loc[:, all_label_unique[all_label_counts == 400]].copy()
#     label_cnt_dict = dict(zip(all_label_unique[all_label_counts == 400], \
#                                     np.full((len(all_label_unique[all_label_counts == 400])), 400)))

#     # Compute mean & variance for each stimulus
#     rate_sorted_mean, rate_sorted_var = compute_mean_var_trial(label_cnt_dict, rate_sorted)
#     rate_sorted_mean_coll, rate_sorted_var_coll = compute_mean_var_trial_collapse(label_cnt_dict, rate_sorted)

#     # Calculate and collect linear slopes for all stimuli
#     slopes = np.zeros((2, rate_sorted_mean_coll.shape[1]))
#     for trial_type_ind, trial_type in enumerate(rate_sorted_mean_coll.columns):
            
#         bool_mean_notzero = rate_sorted_mean_coll.loc[:, trial_type] > 0
#         popt = np.polyfit(np.log10(rate_sorted_mean.loc[bool_mean_notzero, trial_type].values).flatten().astype(np.float32), \
#                             np.log10(rate_sorted_var.loc[bool_mean_notzero, trial_type].values).flatten().astype(np.float32), 1)
#         # popt = np.polyfit(np.log10(rate_sorted_mean.loc[:, trial_type].values).flatten().astype(np.float32), \
#         #                     np.log10(rate_sorted_var.loc[:, trial_type].values).flatten().astype(np.float32), 1)
        
#         slopes[:, trial_type_ind] = popt.copy()
            
#     list_slopes_an_loglog_12[sess_ind] = slopes.copy()

# # Histogram of slopes (all sessions)
# # fig, ax = plt.subplots(figsize=(5, 4))

# list_slopes_an_loglog_flattened = np.concatenate([slopes[0, :].flatten() for slopes in list_slopes_an_loglog_12 if slopes is not None])
# # list_slopes_an_loglog_flattened = np.concatenate((list_slopes_an_loglog_flattened[:1537], list_slopes_an_loglog_flattened[1538:]))

# bin_size = 0.01
# lower_bound, upper_bound = math.floor(np.min(list_slopes_an_loglog_flattened)), \
#     math.ceil(np.max(list_slopes_an_loglog_flattened))

# weights=np.ones_like(list_slopes_an_loglog_flattened)/len(list_slopes_an_loglog_flattened)
# axes[0].hist(list_slopes_an_loglog_flattened, bins=np.arange(lower_bound, upper_bound, bin_size), range=(lower_bound, upper_bound), \
#         weights=weights, color='cornflowerblue')
# # plt.axvline(1, color='r', linestyle='--')

# axes[0].set_xlabel('log(mean)-log(var) slope', fontsize=20)
# # if area_ind == 0 or area_ind == 3:
# axes[0].set_ylabel('Frequency of stimuli', fontsize=20)
# axes[0].set_xticks([0, 0.5, 1, 1.5, 2])
# # axes[0].set_yticks(np.arange(5, 40, 5))
# axes[0].tick_params('both', labelsize=18)

# # Wilcoxon signed-rank test
# wilc = list(wilcoxon(list_slopes_an_loglog_flattened, np.ones_like(list_slopes_an_loglog_flattened), alternative='greater'))
# # plt.annotate(f'p = {wilc[1]:.3f}', xy=(0.8, 0.9), xycoords='axes fraction')

# print(f'slope median = {np.median(list_slopes_an_loglog_flattened):.2f}')

# plt.subplots_adjust(hspace=0.4, wspace=0.3)
# plt.show()

In [ ]:
# with open('SVM_prerequisite_variables.pickle', 'wb') as f:
#     pickle.dump({'tree_variables': ['list_rate_w1', 'list_stm_w1', 'list_neu_loc', 'list_wfdur',
#                                     'list_slopes_an_loglog', 'list_slopes_an_loglog_12'],
#                                     'list_rate_w1': list_rate_w1, 'list_stm_w1': list_stm_w1,
#                                     'list_neu_loc': list_neu_loc, 'list_wfdur': list_wfdur,
#                                     'list_slopes_an_loglog': list_slopes_an_loglog, 'list_slopes_an_loglog_12': list_slopes_an_loglog_12}, f)